In [4]:
# Documentation: https://www.prosper.com/Downloads/Services/Documentation/ProsperDataExport_Details.html
import ml
reload(ml)
from ml import *
import timeit
import scipy
import operator
import numpy as np
import pandas as pd
import cPickle as pickle
from scipy import stats
import seaborn as sns
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt
from pprint import pprint
from __future__ import division
import statsmodels.api as sm
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
plt.style.use('classic')
%matplotlib inline

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

'''
    Analysis originaly performed in Python 2 (deprecated)
    Seaborn, Statsmodel, and * imports broken in Python 3
'''

## Methods

In [5]:
def cov(arr):
    return np.std(arr) / np.mean(arr) 

def fundingTime(arr):
    return (arr.max() - arr.min()).days

def interEventTime(arr):
    return [(t - s).seconds for s, t in zip(sorted(arr), sorted(arr)[1:])]

def avgInterEventTime(arr):
    return np.mean(interEventTime(arr))

def covInterEventTime(arr):
    varr = interEventTime(arr)
    return np.std(varr) / np.mean(varr) 

# projectsgeq4 = np.loadtxt('projectsgeq4contributions.out', delimiter=',', dtype=str)
projectsall = np.loadtxt('projectsall.out', delimiter=',', dtype=str)

## Bids

In [6]:
bids = pd.read_csv("../Data/bid_notick.txt", sep="|")
# Filter winning bids
bids = bids[bids.Status.isin(["Winning", "Partially Participating"])]
# Remove current listings
bids = bids[bids.ListingStatus.isin(["Expired", "Completed"])]
# Filter relevant columns
bids = bids[["ListingKey", "Bid_Key", "MemberKey", "CreationDate", "Amount", "ListingStatus"]]
# Convert creation date to datetime format
bids["CreationDate"] = pd.to_datetime(bids.CreationDate)
# Variable /YYYY/MM to allow monthly time unit analysis
bids["time_trunc"] = bids.CreationDate.map(lambda x: x.strftime('%Y%m'))
# Rename status colunmn for consistency with other projects
bids = bids.rename(index=str, columns={"ListingStatus": "Status"})
# Exclude these bids in preliminary analysis to correspond to final listing list
with open("../Data/final_listings", 'rb') as f:
    filtered = pickle.load(f)
bids = bids[bids['ListingKey'].isin(filtered)] 
# Show sample data
bids.sample(5)

,ListingKey,Bid_Key,MemberKey,CreationDate,Amount,Status,time_trunc
3168266,58F3340712108260398087C,98D03511941659900E79350,D3673365635889466BF71CE,2007-12-09 16:23:58,50.000,Completed,200712
4805424,C75434230297800314FEC1D,33B33527059886197AC6F1A,65853421650794979EAD429,2008-06-04 17:07:54,200.000,Completed,200806
620113,AE82337608201147223E090,94203376918829504E8AA5A,AB8A33757702866398E1684,2006-12-28 15:18:45,50.000,Completed,200612
5679279,293F343122840652713F8EC,39673535062027670032BB3,817E3365718197924D14FEF,2008-09-02 13:01:52,50.000,Expired,200809
5510890,5BB03427648191071ED0BB7,DC0B3532708130274D3B450,126733817665981219B7C51,2008-08-17 21:28:19,50.000,Completed,200808


In [7]:
print "Contributors:", bids.MemberKey.nunique()

Contributors: 53768


In [8]:
print len(bids)

2889147


In [9]:
print len(projectsall)

143770


### Filter Bids

In [10]:
def filter_bids_by_amount(threshold):
    bids_ = []
    threshold = threshold
#     for campaign_id in tqdm_notebook(projectsgeq4):
    for listing_key in tqdm_notebook(projectsall):
        var_df = bids.query("ListingKey == '{0}'".format(listing_key)).sort_values(by='CreationDate')
        series = var_df['Amount'].fillna(0)
        norm_cdf = np.cumsum(series)
        try:
            limit = max(series)*threshold
            if norm_cdf[0] > limit:
                pass
            else:
                 for i in range(len(norm_cdf)):
                    if limit < norm_cdf[i]:
                        bids_.append(var_df[:i-1])  
                        break
        except:
            pass
        
    return pd.concat(bids_, axis=0)

In [11]:
def filter_bids(threshold):  
    bids_ = []
    threshold = threshold
    for listing_key in tqdm_notebook(projectsgeq4):
        var_df = bids.query("ListingKey == '{0}'".format(listing_key)).sort_values(by='CreationDate')
        bids_.append(var_df[:int(len(var_df)*threshold)])
    return pd.concat(bids_, axis=0)

In [12]:
# bids_25 = filter_bids_by_amount(0.25)
# bids_50 = filter_bids_by_amount(0.5)
# bids_75 = filter_bids_by_amount(0.75)
# bids_100 = filter_bids_by_amount(1)

In [13]:
# bids_25.to_csv("../Data/bids_amount_25.csv", index=False)
# bids_50.to_csv("../Data/bids_amount_50.csv", index=False)
# bids_75.to_csv("../Data/bids_amount_75.csv", index=False)
# bids_100.to_csv("../Data/bids_amount_100.csv", index=False)

In [14]:
bids_25 = pd.read_csv("../Data/bids_amount_25.csv")
bids_50 = pd.read_csv("../Data/bids_amount_50.csv")
bids_75 = pd.read_csv("../Data/bids_amount_75.csv")
bids_100 = pd.read_csv("../Data/bids_amount_100.csv")

In [15]:
len(bids_25), len(bids_50), len(bids_75), len(bids_100)

(28432, 81254, 132260, 189173)

### Bids Aggregation

In [16]:
f = {'ListingKey': ['max'],
     'MemberKey':['count'], # Contributor Count
     'Bid_Key': ['count'], # Contributions Count
     'CreationDate': ['min', avgInterEventTime, covInterEventTime], # First Contribution Timestamp, AvgInterEventTime, CovInterEventTime
     'Amount': ['min', 'mean', 'max', cov] # Minimum, Mean, Maximum, CoV
}

#### Bids 25%

In [17]:
bids_25 = bids_25.drop(['time_trunc'], axis=1)
bids_25.CreationDate = pd.to_datetime(bids_25.CreationDate)
bids_25 = pd.DataFrame(bids_25.groupby('ListingKey').agg(f).as_matrix())
bids_25 = bids_25.rename(index=str, columns={0: "NumContributors", 
                                       1: "FirstContributionTimestamp", 
                                       2: "AvgInterEventTime_sec", 
                                       3: "CovInterEventTime", 
                                       4: "MinContributionAmount",
                                       5: "MeanContributionAmount",
                                       6: "MaxContributionAmount", 
                                       7: "CovContributionAmount",
                                       8: "NumContributions",
                                       9: "ProjectID"})
bids_25.head()

,NumContributors,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,NumContributions,ProjectID
0,2,2008-06-18 19:53:23,1.000,0.000,50.000,50.000,50.000,0.000,2,00003423466422120D65CA3
1,1,2007-07-04 06:44:00,NaN,NaN,50.000,50.000,50.000,0.000,1,000433924977359980D3229
2,1,2007-10-11 14:35:08,NaN,NaN,50.000,50.000,50.000,0.000,1,0004340243198378700FF1F
3,7,2007-10-06 09:14:15,37710.000,0.447,50.000,74.854,100.000,0.307,7,000D34012867281790A3888
4,7,2007-09-05 16:15:58,1.000,1.528,50.000,64.286,100.000,0.351,7,00153399719267548BE59C1


#### Bids 50%

In [18]:
bids_50 = bids_50.drop(['time_trunc'], axis=1)
bids_50.CreationDate = pd.to_datetime(bids_50.CreationDate)
bids_50 = pd.DataFrame(bids_50.groupby('ListingKey').agg(f).as_matrix())
bids_50 = bids_50.rename(index=str, columns={0: "NumContributors", 
                                       1: "FirstContributionTimestamp", 
                                       2: "AvgInterEventTime_sec", 
                                       3: "CovInterEventTime", 
                                       4: "MinContributionAmount",
                                       5: "MeanContributionAmount",
                                       6: "MaxContributionAmount", 
                                       7: "CovContributionAmount",
                                       8: "NumContributions",
                                       9: "ProjectID"})
bids_50.head()

,NumContributors,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,NumContributions,ProjectID
0,4,2008-06-18 19:53:23,2.667,0.884,50.000,50.000,50.000,0.000,4,00003423466422120D65CA3
1,1,2007-01-03 09:11:14,NaN,NaN,100.000,100.000,100.000,0.000,1,000433785890431972B4743
2,3,2007-07-04 06:44:00,0.500,1.000,50.000,50.000,50.000,0.000,3,000433924977359980D3229
3,1,2007-10-11 14:35:08,NaN,NaN,50.000,50.000,50.000,0.000,1,0004340243198378700FF1F
4,10,2007-10-06 09:14:15,39276.000,0.504,50.000,74.898,100.000,0.297,10,000D34012867281790A3888


#### Bids 75%

In [19]:
bids_75 = bids_75.drop(['time_trunc'], axis=1)
bids_75.CreationDate = pd.to_datetime(bids_75.CreationDate)
bids_75 = pd.DataFrame(bids_75.groupby('ListingKey').agg(f).as_matrix())
bids_75 = bids_75.rename(index=str, columns={0: "NumContributors", 
                                       1: "FirstContributionTimestamp", 
                                       2: "AvgInterEventTime_sec", 
                                       3: "CovInterEventTime", 
                                       4: "MinContributionAmount",
                                       5: "MeanContributionAmount",
                                       6: "MaxContributionAmount", 
                                       7: "CovContributionAmount",
                                       8: "NumContributions",
                                       9: "ProjectID"})
bids_75.head()

,NumContributors,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,NumContributions,ProjectID
0,5,2008-06-18 19:53:23,672.000,1.725,50.000,50.000,50.000,0.000,5,00003423466422120D65CA3
1,1,2007-01-03 09:11:14,NaN,NaN,100.000,100.000,100.000,0.000,1,000433785890431972B4743
2,5,2007-07-04 06:44:00,24165.750,1.398,50.000,60.000,100.000,0.333,5,000433924977359980D3229
3,1,2007-10-11 14:35:08,NaN,NaN,50.000,50.000,50.000,0.000,1,0004340243198378700FF1F
4,1,2008-06-05 22:22:44,NaN,NaN,50.000,50.000,50.000,0.000,1,00083422661625108817246


#### Bids 100%

In [20]:
bids_100 = bids_100.drop(['time_trunc'], axis=1)
bids_100.CreationDate = pd.to_datetime(bids_100.CreationDate)
bids_100 = pd.DataFrame(bids_100.groupby('ListingKey').agg(f).as_matrix())
bids_100 = bids_100.rename(index=str, columns={0: "NumContributors", 
                                       1: "FirstContributionTimestamp", 
                                       2: "AvgInterEventTime_sec", 
                                       3: "CovInterEventTime", 
                                       4: "MinContributionAmount",
                                       5: "MeanContributionAmount",
                                       6: "MaxContributionAmount", 
                                       7: "CovContributionAmount",
                                       8: "NumContributions",
                                       9: "ProjectID"})
bids_100.head()

,NumContributors,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,NumContributions,ProjectID
0,5,2008-06-18 19:53:23,672.000,1.725,50.000,50.000,50.000,0.000,5,00003423466422120D65CA3
1,1,2008-06-21 22:38:29,NaN,NaN,50.000,50.000,50.000,0.000,1,00033425227988088FA6752
2,2,2007-01-03 09:11:14,3299.000,0.000,50.000,75.000,100.000,0.333,2,000433785890431972B4743
3,7,2007-07-04 06:44:00,20750.000,1.360,50.000,57.143,100.000,0.306,7,000433924977359980D3229
4,1,2007-10-11 14:35:08,NaN,NaN,50.000,50.000,50.000,0.000,1,0004340243198378700FF1F


## Listings

In [21]:
columns = ["Lst_Key", "AmountFunded", "StartDate", "EndDate", "Status", "BorrowerRate", "ProsperScore", "Duration", "CreditGrade", "DebtToIncomeRatio", "IsBorrowerHomeowner"]
listings = pd.read_csv('../Data/listing.txt', sep="|")
listings = listings[columns]
listings = listings.rename(index=str, columns={"Lst_Key": "ProjectID",
                                               "AmountFunded": "AmountRaised"})
creditmap = {'AA':8, 'A':7, 'B':6, 'C':5, 'D':4, 'E':3, 'HR':2, 'NC':1}
listings['CreditGrade'] = listings.CreditGrade.fillna(9)
listings = listings.applymap(lambda s: creditmap.get(s) if s in creditmap else s)
listings = listings[listings['CreditGrade']<9] # Drop records with no credit information
listings.head(5)

,ProjectID,AmountRaised,StartDate,EndDate,Status,BorrowerRate,ProsperScore,Duration,CreditGrade,DebtToIncomeRatio,IsBorrowerHomeowner
0,C0933365069571441D6651D,0.000,2006-02-25 05:51:10,2006-03-04 05:53:21,Expired,0.120,nan,7,3,0.142,NaN
1,06413364714455220D3A5A5,0.000,2006-03-15 17:17:51,2006-03-15 21:02:32,Withdrawn,0.120,nan,7,4,0.207,NaN
2,9D5E336577848196944F197,0.000,2006-03-15 21:52:45,2006-03-29 21:54:21,Expired,0.080,nan,14,2,0.169,NaN
3,06193365103109718753985,0.000,2006-03-22 11:34:35,2006-03-28 20:48:47,Withdrawn,0.080,nan,7,6,0.357,NaN
4,1C0733660810131582558C3,0.000,2006-03-22 12:23:19,2006-03-23 08:45:24,Withdrawn,0.150,nan,7,4,0.512,NaN


## Merge Data

In [22]:
## Data 25%
data_25 = bids_25.merge(listings, on="ProjectID", how="left")
data_25["TimeToFirstContribution (sec)"] = (pd.to_datetime(data_25.FirstContributionTimestamp) - pd.to_datetime(data_25.StartDate)).dt.seconds
data_25["Duration (days)"] = (pd.to_datetime(data_25.EndDate) - pd.to_datetime(data_25.StartDate)).dt.days
data_25 = data_25[data_25["Duration (days)"] >= 0]
statusmap = {'Completed':1, 'Expired':0}
data_25 = data_25.applymap(lambda s: statusmap.get(s) if s in statusmap else s)
del bids_25
## Data 50%
data_50 = bids_50.merge(listings, on="ProjectID", how="left")
data_50["TimeToFirstContribution (sec)"] = (pd.to_datetime(data_50.FirstContributionTimestamp) - pd.to_datetime(data_50.StartDate)).dt.seconds
data_50["Duration (days)"] = (pd.to_datetime(data_50.EndDate) - pd.to_datetime(data_50.StartDate)).dt.days
data_50 = data_50[data_50["Duration (days)"] >= 0]
data_50 = data_50.applymap(lambda s: statusmap.get(s) if s in statusmap else s)
del bids_50
## Data 75%
data_75 = bids_75.merge(listings, on="ProjectID", how="left")
data_75["TimeToFirstContribution (sec)"] = (pd.to_datetime(data_75.FirstContributionTimestamp) - pd.to_datetime(data_75.StartDate)).dt.seconds
data_75["Duration (days)"] = (pd.to_datetime(data_75.EndDate) - pd.to_datetime(data_75.StartDate)).dt.days
data_75 = data_75[data_75["Duration (days)"] >= 0]
data_75 = data_75.applymap(lambda s: statusmap.get(s) if s in statusmap else s)
del bids_75
## Data 100%
data_100 = bids_100.merge(listings, on="ProjectID", how="left")
data_100["TimeToFirstContribution (sec)"] = (pd.to_datetime(data_100.FirstContributionTimestamp) - pd.to_datetime(data_100.StartDate)).dt.seconds
data_100["Duration (days)"] = (pd.to_datetime(data_100.EndDate) - pd.to_datetime(data_100.StartDate)).dt.days
data_100 = data_100[data_100["Duration (days)"] >= 0]
data_100 = data_100.applymap(lambda s: statusmap.get(s) if s in statusmap else s)
del bids_100
# data.head()

## Model Evaluation

In [23]:
features_5 = ["NumContributors", "CovInterEventTime", "CovContributionAmount", "TimeToFirstContribution (sec)", 'Duration (days)']
# Initialize Standard Scaler
scaler = StandardScaler() 

### Bids 25%

In [24]:
X = data_25[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_25['Status'].as_matrix()
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({1: 8662, 0: 3531})
Completed QDA in 0.0493122 seconds
Completed CART in 0.1311969 seconds
Completed GNB in 0.0351257 seconds
Completed RF in 28.6254133 seconds
Completed LR in 0.0787541 seconds
Completed ADB in 1.7648885 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.585,0.851,0.504,0.633,0.644
1,CART,0.652,0.763,0.739,0.751,0.588
2,GNB,0.593,0.837,0.530,0.649,0.639
3,RF,0.628,0.821,0.609,0.699,0.642
4,LR,0.570,0.879,0.458,0.602,0.652
5,ADB,0.711,0.711,0.998,0.831,0.503


In [36]:
classifier = classifiers.get("RF")
estimates = []
X = data_25[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_25['Status'].as_matrix()
for x in tqdm_notebook(range(0,100)):
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "N", len(y)

Mean AUC 0.6456350310123856
St. Dev 0.002728637906439208
N 12193


### Bids 50%

In [25]:
X = data_50[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_50['Status'].as_matrix()
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({1: 17843, 0: 10408})
Completed QDA in 0.0852991 seconds
Completed CART in 0.4082108 seconds
Completed GNB in 0.0679834 seconds
Completed RF in 60.1014798 seconds
Completed LR in 0.2067463 seconds
Completed ADB in 3.5267687 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.612,0.805,0.508,0.623,0.649
1,CART,0.630,0.718,0.682,0.700,0.612
2,GNB,0.619,0.797,0.532,0.638,0.650
3,RF,0.641,0.822,0.551,0.660,0.673
4,LR,0.563,0.865,0.366,0.514,0.634
5,ADB,0.689,0.730,0.806,0.766,0.647


In [35]:
classifier = classifiers.get("RF")
estimates = []
X = data_50[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_50['Status'].as_matrix()
for x in tqdm_notebook(range(0,100)):
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "N", len(y)

Mean AUC 0.6740735742723609
St. Dev 0.0016001533813119271
N 28251


### Bids 75%

In [26]:
X = data_75[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_75['Status'].as_matrix()
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({1: 21867, 0: 14459})
Completed QDA in 0.1124834 seconds
Completed CART in 0.5284735 seconds
Completed GNB in 0.1000756 seconds
Completed RF in 75.532084 seconds
Completed LR in 0.280151 seconds
Completed ADB in 4.648097 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.633,0.789,0.532,0.636,0.658
1,CART,0.638,0.710,0.676,0.692,0.629
2,GNB,0.641,0.774,0.571,0.657,0.659
3,RF,0.659,0.823,0.552,0.661,0.686
4,LR,0.581,0.874,0.355,0.505,0.639
5,ADB,0.694,0.743,0.751,0.747,0.679


In [34]:
classifier = classifiers.get("RF")
estimates = []
X = data_75[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_75['Status'].as_matrix()
for x in tqdm_notebook(range(0,100)):
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "N", len(y)

Mean AUC 0.6872360004747118
St. Dev 0.0012506671613461834
N 36326


### Bids 100%

In [27]:
X = data_100[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_100['Status'].as_matrix()
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({1: 24290, 0: 20484})
Completed QDA in 0.1296583 seconds
Completed CART in 0.7358023 seconds
Completed GNB in 0.2081029 seconds
Completed RF in 106.1038224 seconds
Completed LR in 0.4566875 seconds
Completed ADB in 7.0169904 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.664,0.751,0.571,0.649,0.673
1,CART,0.660,0.694,0.667,0.680,0.659
2,GNB,0.675,0.738,0.623,0.676,0.680
3,RF,0.696,0.827,0.557,0.665,0.709
4,LR,0.615,0.875,0.340,0.489,0.641
5,ADB,0.720,0.742,0.743,0.743,0.718


In [33]:
classifier = classifiers.get("RF")
estimates = []
X = data_100[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_100['Status'].as_matrix()
for x in tqdm_notebook(range(0,100)):
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "N", len(y)

Mean AUC 0.7091187919495191
St. Dev 0.0009304767140333035
N 44774
